In [3]:
import tensorflow as tf
import torch
import numpy as np
import random
from matplotlib import pyplot as plt

2024-04-29 16:53:34.751459: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def get_btch(btch_sz):
    ix = random.choices(np.arange(length - btch_sz),k = (btch_sz))
    tmp1 = [data[i:i+ws] for i  in ix]
    tmp2 = [data[i+1:i+ws+1] for i  in ix]
    x = tf.stack(tmp1)
    y = tf.stack(tmp2)
    return len(tmp1)
  

In [ ]:
def data_set(inp_dt,btch_size):
    dataset = tf.data.Dataset.from_tensor_slices(inp_dt)
    dataset = dataset.window(ws+1,shift = 1,drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(ws+1))
    dataset = dataset.map(lambda window: (window[:ws], window[1:ws+1]))
    dataset = dataset.shuffle(10)
    dataset = dataset.batch(btch_size).prefetch(1)
    for i in dataset:
       
        yield i

In [ ]:
class BigramLanguageModel(tf.keras.Model):
    def __init__(self, vocab_size):
        super(BigramLanguageModel, self).__init__()
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size,embd_sz)
        self.post_embedding_table = tf.keras.layers.Embedding(ws,embd_sz)
        self.head = tf.keras.layers.Dense(vcb_sz,activation = 'linear')
    def call(self, inputs, targets=None):
        tok_embds = self.token_embedding_table(inputs)
        pos_embds = self.post_embedding_table(np.arange(8))
        x = tok_embds+pos_embds
        logits = self.head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = tf.reshape(logits, (B*T, C))
            targets = tf.reshape(targets, (B*T,))
            loss = tf.keras.losses.sparse_categorical_crossentropy(targets, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
            
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(inputs)
            logits = logits[:, -1, :]
            probs = tf.nn.softmax(logits, axis=-1)
            idx_next = tf.random.categorical(probs, 1,dtype=tf.int32)
            
            inputs = tf.concat([inputs, idx_next], axis=1)
            
        return inputs

# vocab_size = 66
# model = BigramLanguageModel(vocab_size)

# # Initialize optimizer
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# # Define your training loop
# batch_size = 32
# for steps in range(1):  # Increase number of steps for good results...
#     # Sample a batch of data
#     xb, yb = get_batch('train')

#     with tf.GradientTape() as tape:
#         logits, loss = model(xb, yb)

#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# print(loss.numpy())


In [ ]:
with open ('input.txt','r',encoding ='utf-8' ) as f1:
    text = f1.read()
print(text[0:100])

In [ ]:
global vcb_sz,ws,length,embd_sz
chars = sorted(list(set(text)))
print(''.join(chars))
vcb_sz = len(chars)
embd_sz = 32
ws = 8
btch_sz_tr = 4
btch_sz_tst = 1

In [ ]:
stoi = {ch : i for  i,ch in enumerate(chars)}
itos = {i : ch for  i,ch in enumerate(chars)}
encode = lambda str1 : [stoi[j] for j in str1]
decode = lambda lst2 : ''.join([itos[k] for k in lst2])
print(encode('hiithere'))
print(decode([45, 46, 46, 57, 45,0, 42, 55, 42]))


In [ ]:

data = tf.constant(encode(text),dtype = tf.int32)
length = len(data)
print(length)

In [ ]:

splt = int(0.85*len(data))
tr_dt = data[:splt]
tst_dt = data[splt+1:]
tst_dt.shape

In [ ]:
get_btch(btch_sz_tr)

In [ ]:
xb[0]

In [ ]:

#smp_dt = tf.reshape(smp_dt,shape = (btch_sz*ws,vcb_sz))
model = BigramLanguageModel(vcb_sz)

# Initialize optimizer
optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-3)

# Define your training loop
#batch_size = 32
for steps in range(2000):  # Increase number of steps for good results...
    # Sample a batch of data
#     for i in model.trainable_variables:
#         print(i)
    xb, yb = get_btch(btch_sz_tr)

    with tf.GradientTape() as tape:
        logits, loss = model(xb,yb)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    print(loss.numpy(),steps)
print(decode(model.generate(inputs = tf.zeros((1, 1),dtype=tf.int32), max_new_tokens=1000).numpy().squeeze().tolist()))


In [ ]:
# in_arr = get_btch(1)

# trg = tf.constant([0,1,2,3,4,5,6,7],shape=(1,8))
# inp1 = tf.keras.Input(shape = (8))
# inp2 = tf.keras.Input(shape = (8))
# x = tf.keras.layers.Embedding(vcb_sz,32)(inp1)
# y = tf.keras.layers.Embedding(ws,32)(inp2)
# otpt = tf.keras.layers.Dense(66,activation = 'linear')(x+y)
# md = tf.keras.Model(inputs = [inp1,inp2],outputs = [otpt])
# md.compile(tf.keras.optimizers.AdamW(),tf.keras.losses.SparseCategoricalCrossentropy())
# md.fit([in_arr[0],trg],in_arr[1])

In [ ]:
# x = np.linspace(0,1,66)
# y = np.linspace(0,1,8)
# print(x.shape)
# X,Y = np.meshgrid(x,y)
# print(X.shape)
# #levels = np.linspace(np.min(z), np.max(z), 25)
# #fig , ax = plt.subplots(1,1)
# levels = np.linspace(np.min(asd[0]), np.max(asd[0]), 25)
# fig , ax = plt.subplots(1,1)
# ax.contour(X,Y,asd[0])
# fig,bx = plt.subplots(1,2)
# bx = plt.axes(projection ='3d')

# bx.plot_surface(X,Y,asd[0])

In [ ]:
documentv=([1,2,3],'fg',(34,56))

In [ ]:
documentv[-1].append('Poetry')

In [ ]:
x1 = [[2,1],[3,1],[4,1]]
c = np.mean(x1,axis =1)
d = x1 - c
v = np.transpose(d)
f = np.matmul(v,d)
print(d.shape,v.shape)
f
#x4 = tf.stack([x1,x2],axis = 0)

In [ ]:
x1 = [[2,3,4],[1,1,1]]
c = np.mean(x1,axis =0)

d = x1 - c
v = np.transpose(d)
f = np.matmul(v,d)
print(d.shape,v.shape)

In [ ]:
f

In [ ]:
asd = lambda x : x**2
asd(200)

In [ ]:
x2 = np.array([1,2,3,4])
x3 = np.array([1,0,1,2])

mat = (np.stack([x2,x3],axis =1 ))
print(mat)
mat  = np.transpose(mat)
c = np.mean(mat,axis =0)

d = (mat - c)
v = np.transpose(d)
f = np.matmul(v,d)
f/3


In [ ]:
exp1 = (np.mean(x2*x2) - np.mean(x2)**2)/2

In [ ]:
exp1

In [ ]:
np.sqrt(1.6669)

In [20]:
SD = np.array([[[99, 83, 22, 85],
[86, 64, 30, 26],
[54, 98, 18, 92],
[63, 63, 38, 74]],
[[48, 57, 18, 88],
[84, 44, 13, 44],
[73, 65, 38, 45],
[89, 68, 44, 49]],
[[75, 77, 97, 44],
[88, 69, 50, 68],
[64, 59, 53, 45],
[34, 42, 92, 86]]])

In [24]:
SD[1,0:,2]


array([18, 13, 38, 44])

In [ ]:
89//2

In [ ]:
for i in [10, 20]:
    for j in [1, 2]:
        print(i, j)    

In [ ]:
12345//10

In [ ]:
12345%10

In [ ]:
1%2

In [ ]:
44//2

In [15]:
num = [123,45,67,89,90,11,34,5678,98756]
prm = []
for nums in num:
    for i in range(2,nums//2):
        if nums%i == 0:
           
            break
    else :
        prm.append(nums)

        
        
    

In [9]:
txt = int(input('enetr a number'))
total = 0
quo = 10000
while quo != 0:
    quo = txt//10
    rem = txt%10
    total += rem
    txt = quo
    
    

enetr a number1234567891


In [10]:
total

46

In [12]:
arr = np.array([123,45,67,89,90,11,34,5678,98756])
print(arr[::4])  

[  123    90 98756]


In [17]:
np.trunc(3.45)

3.0